In [ ]:
import micropip
await micropip.install('folium')

In [ ]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Filepaths
fp_sdBeats = "data/pd_neighborhoods_datasd.geojson"
fp_force = "data/ripa_force_actions_datasd.csv"
fp_sdStops = "data/ripa_stops_datasd_2004_2005.csv"
fp_sdbeatsNames = "data/Preview_Beat_Matching_Table__Final_Corrections_Applied_numbers.csv"

# Map of San Diego

In [ ]:
gdf_sdBeats = gpd.read_file(fp_sdBeats)
gdf_sdBeats 

In [ ]:
gdf_sdBeats.columns

In [ ]:
# Get center of map
center = [gdf_sdBeats .geometry.centroid.y.mean(), gdf_sdBeats .geometry.centroid.x.mean()]

# Create folium map
mapSD = folium.Map(location=center, zoom_start=11)

# Add polygons
#folium.GeoJson(gdf).add_to(m)
folium.GeoJson(
                gdf_sdBeats ,
                name="Neighborhoods",
                tooltip=folium.GeoJsonTooltip(fields=["name"], 
                aliases=["Neighborhood:"])
).add_to(mapSD)
mapSD

In [ ]:
df_beatFix = pd.read_csv(fp_sdbeatsNames)
# Drop Unnamed: 0.1	Unnamed: 0
#df_beatFix = df_beatFix.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])
df_beatFix

In [ ]:
# View entire dataframe columns and rows
# merge beat fix and gdf_sdBeats
gdf_sdBeats = gdf_sdBeats.merge(df_beatFix, left_on="name", right_on="beatNameFromGeo", how="left")
gdf_sdBeats

In [ ]:
# undo set_option to default
print(gdf_sdBeats["name"].unique().tolist())

# Use of Force Data

In [ ]:
# Read and preprocess the force actions data
df_force = pd.read_csv(fp_force)
# excract the year from the 'date' column
df_force["year"] = pd.to_datetime(df_force["insertDatetime"]).dt.year 
df_force

In [ ]:
df_force["forceactiontaken"].value_counts()

In [ ]:
# Bar chart excluding forceactiontaken
df_force["forceactiontaken"].value_counts().plot(kind="bar", figsize=(10, 6), title="Force Actions Taken")
plt.xlabel("Force Action Taken")
plt.ylabel("Count")
plt.xticks(rotation=90)
plt.grid(True)
plt.tight_layout()
plt.show()

# Intermediate Data: Police Stops

In [ ]:
df_stops = pd.read_csv(fp_sdStops)
df_stops 

In [ ]:
df_stops.columns

In [ ]:
df_stops["beat"].unique()

In [ ]:
df_stops["beatName"].unique()

In [ ]:
# Strip number off beat names and create a new column
df_stops["beatName"] = df_stops["beatName"].fillna("Unknown")
df_stops["beatNumber"] = df_stops["beatName"].str.extract(r'^(\d+)').astype("Int64")
df_stops

# Merging Data to Tie it Altogether

In [ ]:
# Merge stops with beat names on beatNumber
df_stops = df_stops.merge(df_beatFix, left_on="beatNumber", right_on="beatNumber", how="left")
df_stops

In [ ]:
# Merge df_force and df_stops oon id
df_mergedStopsForce = pd.merge(df_force, df_stops, on="id", how="inner")
df_mergedStopsForce

In [ ]:
df_mergedStopsForce.columns

# Analysis

In [ ]:
# Group on beatNameFromGeo and forceactiontaken summing them
df_grouped = df_mergedStopsForce.groupby(["beatNameFromGeo", "forceactiontaken"]).size().reset_index(name="count")
df_grouped

In [ ]:
# create a pivot table
df_pivot = df_grouped.pivot(index="beatNameFromGeo", columns="forceactiontaken", values="count").fillna(0)
# Reset index to make beatNameFromGeo a column
df_pivot


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(df_pivot)

In [ ]:
# merge the gdf_sdBeats with df_mergedStopsForce on beat number
gdf_mergedStopsForce = gdf_sdBeats.merge(df_grouped, left_on="name", right_on="beatNameFromGeo", how="left")
gdf_mergedStopsForce

# Map it

In [ ]:
# Fill missing values
gdf_mergedStopsForce["forceactiontaken"] = gdf_mergedStopsForce["forceactiontaken"].fillna("Unknown")
gdf_mergedStopsForce["count"] = gdf_mergedStopsForce["count"].fillna(0)

# Aggregate force actions and counts per beat with line breaks
tooltip_df = (
    gdf_mergedStopsForce
    .groupby(['beatNumber', 'beatNameFromGeo_x'])
    .apply(lambda x: '<br>'.join(f"{a}: {int(c)}" for a, c in zip(x['forceactiontaken'], x['count'])))
    .reset_index(name='force_summary')
)

# Merge tooltip data into one row per beat polygon
gdf_tooltip = (
    gdf_mergedStopsForce
    .drop_duplicates(subset='beatNumber')
    .merge(tooltip_df, on=['beatNumber', 'beatNameFromGeo_x'], how='left')
)

# Create folium map
mapForce = folium.Map(location=center, zoom_start=11)

# Add GeoJson layer with tooltip
folium.GeoJson(
    gdf_tooltip,
    name="Force Actions",
    tooltip=folium.GeoJsonTooltip(
        fields=["beatNameFromGeo_x", "force_summary"],
        aliases=["Beat Name:", "Force Actions:"],
        sticky=True,
        labels=True
    )
).add_to(mapForce)

mapForce
